In [ ]:
# download model weights
!curl -L "https://drive.usercontent.google.com/download?id=1dlwaElu0dQQdoEeJkuP2LKGx1TSCjE-z&confirm=xxx" --output yolopx.pth

In [38]:
PATH = "yolopx.pth"
OUTPUT_PATH = "yolopx.onnx"

import torch
from lib.config import cfg
from lib.models import get_net
from lib.utils.utils import select_device

# load model
logger = None
device = select_device(logger, '0')
model = get_net(cfg)
checkpoint = torch.load(PATH, map_location=device)
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)

In [ ]:
img_size = 640
img = torch.zeros((1, 3, img_size, img_size), device=device, dtype=torch.float32)

torch.onnx.export(
    model,
    img,
    f = OUTPUT_PATH,
    input_names = ["input"],
    output_names = ["det_out", "det_out_dim0", "det_out_dim1", "det_out_dim2", "drive_area_seg", "lane_line_seg"],
    dynamo = False # some issues with dynamo with this model
)

In [40]:
# check model
import onnx
model = onnx.load(OUTPUT_PATH)
onnx.checker.check_model(model)